In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
table = dbutils.widgets.get("table")

vs_endpoint_name = dbutils.widgets.get("vs_endpoint_name")
vs_index_name = dbutils.widgets.get("vs_index_name")
pipeline_type = dbutils.widgets.get("pipeline_type")
primary_key = dbutils.widgets.get("primary_key")
embedding_source_column = dbutils.widgets.get("embedding_source_column")
embedding_model_endpoint_name = dbutils.widgets.get("embedding_model_endpoint_name")

table_path = f"{catalog}.{schema}.{table}"
index_path = f"{catalog}.{schema}.{vs_index_name}"
display(f"Reading table from path: {table_path}")
display(f"Reading index from path: {index_path}")

In [0]:
def index_exists(vsc, endpoint_name, index_full_name):
    try:
        vsc.get_index(endpoint_name, index_full_name).describe()
        return True
    except Exception as e:
        if 'RESOURCE_DOES_NOT_EXIST' not in str(e):
            print(f'Unexpected error describing the index. This could be a permission issue.')
            raise e
    return False

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

In [0]:
if not index_exists(vsc, vs_endpoint_name, index_path):
    vsc.create_delta_sync_index_and_wait(
        endpoint_name=vs_endpoint_name,
        index_name=index_path,
        primary_key=primary_key,
        source_table_name=table_path,
        pipeline_type=pipeline_type,
        embedding_source_column=embedding_source_column,
        embedding_model_endpoint_name=embedding_model_endpoint_name,
        verbose=True
    )
    print(f"Created index {index_path}.")
else:
    vsc.get_index(endpoint_name=vs_endpoint_name, index_name=index_path)
    print(f"Synced index: {vs_index_name}.")

In [0]:
question = "What dish can I prepare if I just have bananas and bread?"

results = vsc.get_index(vs_endpoint_name, index_path).similarity_search(
  query_text=question,
  columns=["ingredients", "directions"],
  num_results=1)
source = results.get('result', {}).get('data_array', [])
display(source)